In [1]:
! pip install tensorboardX

     |████████████████████████████████| 317kB 3.4MB/s 


In [2]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [4]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

In [8]:
for iter_no, batch in enumerate(iterate_batches(
        env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = \
        filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

0: loss=0.692, reward_mean=19.7, rw_bound=22.0
1: loss=0.690, reward_mean=21.6, rw_bound=24.0
2: loss=0.679, reward_mean=19.4, rw_bound=22.0
3: loss=0.674, reward_mean=32.0, rw_bound=40.5
4: loss=0.657, reward_mean=30.5, rw_bound=37.0
5: loss=0.642, reward_mean=32.9, rw_bound=38.5
6: loss=0.629, reward_mean=40.4, rw_bound=44.5
7: loss=0.626, reward_mean=44.1, rw_bound=44.5
8: loss=0.610, reward_mean=54.1, rw_bound=67.5
9: loss=0.591, reward_mean=57.1, rw_bound=67.0
10: loss=0.597, reward_mean=50.6, rw_bound=59.0
11: loss=0.597, reward_mean=59.4, rw_bound=62.0
12: loss=0.591, reward_mean=60.4, rw_bound=72.5
13: loss=0.586, reward_mean=87.1, rw_bound=104.5
14: loss=0.557, reward_mean=61.9, rw_bound=70.5
15: loss=0.560, reward_mean=61.4, rw_bound=72.5
16: loss=0.564, reward_mean=80.7, rw_bound=92.5
17: loss=0.544, reward_mean=82.4, rw_bound=87.0
18: loss=0.542, reward_mean=72.6, rw_bound=75.0
19: loss=0.546, reward_mean=75.2, rw_bound=74.0
20: loss=0.549, reward_mean=94.4, rw_bound=106.0
